# Big Data different
The idea is to joint the tables better

In [ ]:
def Country_population_data(country_code: str, population_data: pd.DataFrame, asylum_data: pd.DataFrame) -> pd.DataFrame:
    country_population = population_data[population_data["Country Code"] == country_code].drop(columns=["Country Name", "Country Code"]).transpose().reset_index()
    country_population.columns = ['year', 'population']
    country_population['year'] = country_population['year'].astype(int)

    country = asylum_data[asylum_data["country_of_origin_abbr"] == country_code].groupby("year", as_index=False)["count"].sum()
    
    country = country.merge(right= country_population, how='left', on='year')
    country.drop(country[country['population'].isna()].index, inplace=True)
    country["percentage_of_population_migration"] = (country["count"] / country["population"]) * 100

    country.rename(columns={'count': "displaced"}, inplace=True)
    country["country_of_origin_abbr"] = country_code
    return country

def Get_country_population_df(pop, asy) -> pd.DataFrame:
    countries = asy["country_of_origin_abbr"].unique()
    countries_not_in_the_analysis = []
    full_data = pd.DataFrame()
    for country in countries:
        if country in pop['Country Code'].values:
            full_data = pd.concat([full_data, Country_population_data(country, population_data=pop, asylum_data=asy)])
        else:
            countries_not_in_the_analysis.append(asylum[asylum['country_of_origin_abbr'] == country]['country_of_origin_name'].iloc[0])
        
    print('These countries were remove from the asylum due to not appear in the WorldBank population dataset:')
    print(countries_not_in_the_analysis)
    return full_data

tt = Get_country_population_df(population, asylum)
tt[tt['country_of_origin_abbr'] == 'ITA']


## check first 
def Country_by_destination(country_abbr: str, asy: pd.DataFrame) -> pd.DataFrame:
    country = asy[asy["country_of_origin_abbr"] == country_abbr]
    country_destination = country.groupby("country_of_asylum_name")["count"].sum().reset_index().sort_values(by="count", ascending=False)
    return country_destination